In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sdv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/

In [ ]:
# importing required libraries
import numpy as np
import pandas as pd

from sdv.single_table import GaussianCopulaSynthesizer
from sdv.metadata import SingleTableMetadata

from sdv.sampling import Condition

In [ ]:
train_data = pd.read_csv('/content/drive/MyDrive/DLC-IDS/Datasets/train_merged_DOS.csv')
print(train_data.label.value_counts())

label
normal    67343
Dos       45927
Other     12703
Name: count, dtype: int64


In [ ]:
attacks = ['Dos']

In [ ]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(train_data)

# Step 1: Create the synthesizer
synthesizer = GaussianCopulaSynthesizer(metadata)

# Step 2: Train the synthesizer
synthesizer.fit(train_data)

/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:119: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/usr/local/lib/python3.11/dist-packages/sdv/single_table/base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [ ]:
# SYNTHETIC DATA GENERATION WITH CONDITION
generated_df = pd.DataFrame(columns=train_data.columns)

for feature in attacks:
  synthetic_data_condition = Condition(
      num_rows = 30000,
      column_values={'label': feature}
  )

  synthetic_data = synthesizer.sample_from_conditions(
      conditions=[synthetic_data_condition]
  )

  print(synthetic_data.label.value_counts())
  generated_df = pd.concat([train_data, synthetic_data]) # Use pd.concat instead of generated_df.concat
  print(generated_df.label.value_counts())

Sampling conditions: 100%|██████████| 30000/30000 [00:12<00:00, 2394.03it/s]

label
Dos    30000
Name: count, dtype: int64
label
Dos       75927
normal    67343
Other     12703
Name: count, dtype: int64


In [ ]:
# saving the dataframe
generated_df.to_csv('/content/drive/MyDrive/DLC-IDS/Datasets/train_merged_DOS_sdv.csv', index = False)